In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


Link to the GSS 2006 Codebook - https://www.thearda.com/Archive/Files/Codebooks/GSS2006_CB.asp

Link to the WVS Codebook- https://www.thearda.com/Archive/Files/Codebooks/WVS2010_CB.asp

In [ ]:

gss_2006 = pd.read_csv('/gdrive/My Drive/Data Analysis GR5015/GSS_2006.csv')
wvs_data = pd.read_csv('/gdrive/My Drive/Data Analysis GR5015/WVS.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (266,318,322,2251,2269,3265,3293,3295,3317,3386,3412,3413,3416,3417,3430,3434,3435,3436,3437,3438,3439,3440,3441,3443,3448,3449,3450,3451,3452,3453,3454,3455,3456,3458,3461,3462,3463,3479,3480,3481,3482,3483,3484,3488,3493,3498,3499,3500,3501,3502,3503,3504,3505,3506,3507,3508,3509,3510,3511,3512,3513,3516,3517,3518,3519,3522,3523,3537,4650,4652,4851,4912,4913,4918,4919,4924,4925,4930,4931,4936,4937,4939,4941,4942,4943,4944,4945,4947,4948,4949,4950,4951,4953,4956,4957,4962,4963,4968,4969,4971,4972,4973,4974,4975,4977,4978,4979,4980,4981,4986,4987,4998,4999,5000,5001,5002,5003,5004,5005,5021,5022,5023,5024,5025,5026,5027,5028,5035,5036,5037,5038,5039,5040,5041,5042,5066,5067,5078,5079) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#renaming columns to convenience and to add context
wvs_data.rename(columns={ 'V2':'country',
                          'V242':'age',
                          'V57':'marital',
                          'V240':'sex',
                          'V78':'environment_imp', # this asks- Is looking after environment important 
                          'V122':'conf_env_org', # this asks- how much confidence do they have in environmental organizations
                          'V82': 'donated_eco_org', # this asks if people have donated to eco orgs
                          'V239':'ses', ## this asks where you place yourself in the income distribution in your country 
                          'V32': 'mem_charitable_orgs'}, ## this asks if theyre a member of charitable orgs
                 inplace=True)

# Recoding
wvs_data['environment_imp']=7-(wvs_data['environment_imp']) ## reverse code environment_imp -  higher the value higher importance
wvs_data['conf_env_org']=5-(wvs_data['conf_env_org']) ## reverse code conf_env_org -  higher the value higher confidence in environmental orgs

# Encoding Marital status variable
conditions = [
    (wvs_data['marital'] ==1) ,
     (wvs_data['marital'] >1 )]
choices = [1,0]
wvs_data['married'] = np.select(conditions, choices, default=np.nan)

# Encoding Sex variable
conditions = [
    (wvs_data['sex'] ==2) ,
     (wvs_data['sex'] !=2 )]
choices = [1,0]
wvs_data['female'] = np.select(conditions, choices, default=np.nan)

# only keeping observations that answered about their SES level
wvs_data = wvs_data.dropna(subset = ["ses"])

# Encoding Environmental Importance- People who say 'Like me' and 'Very Much like me' into 1 and the others in to 0
conditions = [
    (wvs_data['environment_imp'] >= 6) ,
     (wvs_data['environment_imp'] < 6 )]
choices = [1,0]
wvs_data['high_environment_imp'] = np.select(conditions, choices, default=np.nan)

# Encoding Donation to eco orgs
conditions = [
    (wvs_data['donated_eco_org'] ==1) ,
     (wvs_data['donated_eco_org'] >1 )]
choices = [1,0]
wvs_data['donated_eco_org'] = np.select(conditions, choices, default=np.nan)

# Encoding Active member of charitable orgs
conditions = [
    (wvs_data['mem_charitable_orgs'] ==2) ,
     (wvs_data['mem_charitable_orgs'] <2 )]
choices = [1,0]
wvs_data['active_mem_char_orgs'] = np.select(conditions, choices, default=np.nan)

### Question 2
***2. Run a multiple (binary) logistic model.  (It can be the same as the above LPM or -- even better -- a new model.)  Tell me how you think your independent variables will affect your dependent variable.  Interpret your results in the logit scale.  Were your expectations correct?  Why or why not?***



I am looking at variable to see if someone donated_eco_org based on their socio-economic-status and age

#### **Observations & Interpretations**

- From the coefficient we can say that people who are of higher socio-economic-status are more likely to donate to a Ecological Charitable Org. The magnitude is not interperetable in a logit model, that we do by seeing odds ratio which is the next question.

- From the coefficient we can say that with increase in age it is more likely that people donate to a ecological charitable organization.


- And both the above interpretation are statistically significant given the p-valuea are 0.000 an 0.002 (which is much lower than 0.05 i.e a rule of thumb to reject the NULL hypothesis)

#### **My expectation and if it worked out or not**

My intuition was that in USA given the complex and swinging tax laws, along with so many vested interests and mechnaism to evade taxes through charitable fundings etc. and the number of wealthy people, the donation to environmental causes-- which is hot-button poilitical topic, would increase.

And as people grow older they have more savings and better financial planning nad hence will be able to spare some money for donations.

The results were in-line with my intuition.


In [ ]:
logit1 = sm.formula.logit(formula = "donated_eco_org ~ ses + age",subset=((wvs_data['country']==840)), data = wvs_data).fit()
print (logit1.summary()) 

Optimization terminated successfully.
         Current function value: 0.473108
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:        donated_eco_org   No. Observations:                 2158
Model:                          Logit   Df Residuals:                     2155
Method:                           MLE   Df Model:                            2
Date:                Fri, 02 Apr 2021   Pseudo R-squ.:                 0.02695
Time:                        20:55:58   Log-Likelihood:                -1021.0
converged:                       True   LL-Null:                       -1049.2
Covariance Type:            nonrobust   LLR p-value:                 5.267e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.0351      0.241    -12.590      0.000      -3.508      -2.563
ses            0.1958      0.